In [97]:
# ---------------------------------------------------------
# Import declaration.
# ---------------------------------------------------------

import pandas as pd
import numpy as np
from IPython.display import Image
import math

In [92]:
# ----------------------------------------------------------
# Load the data from jester-data-1.csv.
# Let's just choose 100 rows to use(Just because 24981 is too much.)
# ----------------------------------------------------------

#jokes_number = np.linspace(0, 100, 101)
#jokes_number = np.append(jokes_number, "test_set")
#print (jokes_number)
df = pd.read_csv("jester-data-1.csv",header = None)
j_df = df.iloc[0:100,:]
del j_df[j_df.columns[0]]
#j_df.columns = jokes_number
j_df.head()

# ----------------------------------------------------------
# Split data into test_set and training_set based on giving label 99
# Firstly, Copy j_df as j_test_df and j_train_df
# Scondly, Split and label datas simply using 2*for loops and np.random.rand()
# ----------------------------------------------------------

j_shape = j_df.shape
j_df1 = j_df.copy()
j_df2 = j_df.copy()
train_values = j_df1.values # Copy data values from j_df as train_values prepared for next step
test_values = j_df2.values # Copy data values from j_df as test_values prepared for next step

print(j_shape [0])
#print(train_values)


# Label data 99 in train_set and keep that data labeling others 99 in test_set

for i in range (0, j_shape[0]):
    for j in range (0, j_shape[1]):
        if np.random.rand() > 0.9:
            train_values[i,j] = 99
        else:
            test_values[i, j] = 99

print(train_values)
print(test_values)

j_train_df = pd.DataFrame(train_values)
j_test_df = pd.DataFrame(test_values)

print(j_train_df)
print(j_test_df)

# End of splitting data

100
[[-7.82 99.   -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ... 99.   -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 3.3   1.31 99.   ... -0.44  1.17  6.5 ]
 [99.   99.   99.   ... 99.   99.   99.  ]]
[[99.    8.79 99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ...  0.34 99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.    4.37 ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]]
       0      1      2      3      4      5      6      7      8      9   \
0   -7.82  99.00  -9.66  -8.16  -7.52  -8.50  -9.85   4.17  -8.98  -4.76   
1    4.08  -0.29   6.36  99.00  99.00  -9.66  99.00  -5.34   8.88   9.22   
2   99.00  99.00  99.00  99.00   9.03  99.00   9.03   9.27  99.00  99.00   
3   99.00   8.35  99.00  99.00  99.00   8.16  -2.82   6.21  99.00   1.84   
4    8.50   4.61  -4.17  -5.39   1.36   1.60   7.04   4.61  -0.44   5.73   
5   -6.17  -

In [93]:
# -------------------------------------------------------------
# Create a model to infer the hidden rating of users.
# (As it would take a long time for system to train, a better modeling method shoud be arose.)
# Firstly, let's assume how many hidden features do a joke have, and how many hidden standards do a costumer have(2).
# -------------------------------------------------------------

# Let's say the numbers of features of joke & costumer is 2
n_features = 10 

# Then we create 2 lists ofrandom values for both hidden features & standards
reader_ratings = train_values
latent_reader_preferences = np.random.random((j_shape[0], n_features))
latent_jester_features = np.random.random((j_shape[1],n_features))

# Print them so we'll see 24982 readers have 24982*2 hidden standards.
# 101 Jesters have 101*2 hidden features.
print(latent_reader_preferences.shape)
print(latent_jester_features.shape)

(100, 10)
(100, 10)


In [94]:
# -------------------------------------------------------------
# Secondly, create the training fuctions.
# Functions should at least include 2.
# The first one should generate a rating given reader no.and jester no..
# The second one would use all the datas we have to train and give a best results.
# -------------------------------------------------------------

# Predict a rating given a reader_id and a jester_id(with dot).
def predict_rating_dot(reader_id, jester_id):
    reader_preference = latent_reader_preferences[reader_id]
    jester_features = latent_jester_features[jester_id]
    return reader_preference.dot(jester_features)

# Predict a rating given a reader_id and a jester_id(with functions).
def predict_rating_function(reader_id, jester_id):
    reader_preference = latent_reader_preferences[reader_id]
    jester_features = latent_jester_features[jester_id]
    rating = reader_preference[0]*jester_features[0] + reader_preference[1]*jester_features[1]
    return rating

# Caution: two rating generators are absolutely the same!

# Training function.
def train(reader_id, jester_id, rating, alpha = 0.0001):
    
    prediction_rating = predict_rating_dot(reader_id, jester_id)
    err =  ( prediction_rating- rating ); # give a error
    
    reader_pref_values = latent_reader_preferences[reader_id][:] # pref value record.
    
    # Here we update the hidden features & standards.
    latent_reader_preferences[reader_id] -= alpha * err * latent_jester_features[jester_id]
    latent_jester_features[jester_id] -= alpha * err * reader_pref_values
    
    return err

print(predict_rating_dot(0, 0), predict_rating_function(0, 0))

train(0, 0, reader_ratings[0, 0])

print(predict_rating_dot(0, 0), predict_rating_function(0, 0))

5.149837169399869 0.49694315783548415
5.135829073872094 0.4950558631736304


In [114]:
# ----------------------------------------------------------------
# Finally, we train all our data in a loop.
# ----------------------------------------------------------------

# In the loop, we should not use rating value = 99
# Plus, we should predict it also.
# It takes a long time training in 300000 iterations so just take 50 iterations first as test.
def sgd(iterations = 500000):
    prev_mse = 0
    t_mse = 0
    t_number = 0
    for iteration in range(0,iterations):
        error = []
        for reader_id in range(0,latent_reader_preferences.shape[0]):
            for jester_id in range(0,latent_jester_features.shape[0]):
                rating = reader_ratings[reader_id][jester_id]
                if (rating != 99):
                    err = train(reader_id, jester_id, rating)
                    error.append(err)
        predict_train = latent_reader_preferences.dot(latent_jester_features.T)
        predict_test_df =j_test_df.copy()
        predict_test = predict_test_df.values

        for i in range (0, j_shape[0]):
            for j in range (0, j_shape[1]):
                if test_values[i,j] != 99:
                    predict_test[i, j] = predict_train[i, j]
        for i in range (0, j_shape[0]):
            for j in range (0, j_shape[1]):
                t_mse = t_mse + (j_test_df.values[i, j] - predict_test[i, j]) * (j_test_df.values[i, j] - predict_test[i, j])
                if predict_test[i, j] != 99:
                    t_number = t_number + 1
        t_mse = t_mse/t_number
        print(t_mse)
        if abs(t_mse -prev_mse)< 0.001:
            break
        prev_mse = t_mse

In [115]:
# --------------------------------------------------------------------
# Now let's see what would we get.
# Wish we could get our results very fast.(23333333333)
# --------------------------------------------------------------------
sgd()

23.062007193662453
11.548806383446426
7.694631952008727
5.770276252886157
4.6162134651937965
3.847021449355632
3.2976770433890135
2.8857079671185777
2.5653093717355393
2.30900357868641
2.0993071692844305
1.9245657191671333
1.7767114203338459
1.6499819182430837
1.5401517054847085
1.444051788523727
1.359258908018664
1.2838883658046283
1.2164522827131912
1.1557603837132717
1.1008491330435133
1.050930197965032
1.0053523579513812
0.9635729373802316
0.9251360949224493
0.8896561234483727
0.8568044613307533
0.8262994872290119
0.7978984264343605
0.7713908760463093
0.7465935834153105
0.7233462036832726
0.701507828722682
0.6809541286482256
0.6615749833799781
0.6432725089654671
0.6259594039728399
0.6095575569918604
0.5939968683750834
0.5792142487254791
0.5651527639529998
0.5517609024709063
0.538991944647809
0.5268034182468507
0.5151566264757174
0.5040162375975914
0.4933499269340713
0.48312806361928146
0.4733234357119227
0.4639110082949718
0.4548677100351622
0.44617224437100067
0.4378049220763833
0

In [116]:
# -----------------------------------------------------------
# The last step is giving a prediction to train_set and test_set.
# -----------------------------------------------------------
predict_train = latent_reader_preferences.dot(latent_jester_features.T)
predict_test_df =j_test_df.copy()
predict_test = predict_test_df.values

#print(j_test_df)
#print(predict_test)

for i in range (0, j_shape[0]):
    for j in range (0, j_shape[1]):
        if test_values[i,j] != 99:
            predict_test[i, j] = predict_train[i, j]

print(j_test_df)
print(test_values)

       0      1      2      3      4      5      6      7     8      9   ...   \
0   99.00   8.79  99.00  99.00  99.00  99.00  99.00  99.00  99.0  99.00  ...    
1   99.00  99.00  99.00   4.37  -2.38  99.00  -0.73  99.00  99.0  99.00  ...    
2   99.00  99.00  99.00  99.00  99.00   9.27  99.00  99.00  99.0  99.00  ...    
3   99.00  99.00  99.00  99.00   1.80  99.00  99.00  99.00  99.0  99.00  ...    
4   99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.0  99.00  ...    
5   99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.0  -1.80  ...    
6   99.00  99.00  99.00  99.00   8.59  99.00  99.00  99.00  99.0  99.00  ...    
7   99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.0  99.00  ...    
8   99.00  99.00  99.00  99.00  -8.74  99.00  99.00  99.00  99.0  99.00  ...    
9   99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.00  99.0   8.16  ...    
10  99.00  99.00  99.00  99.00  99.00  99.00  99.00  -5.29  99.0  99.00  ...    
11  99.00  99.00  99.00  99.

In [117]:
values = [zip(j_test_df.values[i], predict_test[i]) for i in range(0,predict_test.shape[0])]
comparison_data_test = pd.DataFrame(values)
comparison_data_test.columns = j_df.columns
comparison_data_test.applymap(lambda x: "(%2.3f|%2.3f)"%(x[0],x[1]))
#comparison_data_test

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(99.000|99.000),(8.790|-3.481),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
1,(99.000|99.000),(99.000|99.000),(99.000|99.000),(4.370|4.593),(-2.380|5.261),(99.000|99.000),(-0.730|-2.367),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(0.340|0.732),(99.000|99.000),(99.000|99.000)
2,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(9.270|5.782),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
3,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(1.800|5.077),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
4,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(5.190|4.866),(99.000|99.000),(99.000|99.000),(5.190|1.733),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
5,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(-1.800|-4.567),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
6,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(8.590|4.830),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
7,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
8,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(-8.740|-7.362),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),...,(4.370|-0.442),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)
9,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(8.160|4.779),...,(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000),(99.000|99.000)


In [118]:
#Test data mse.
import math
t_mse = 0
t_number = 0
for i in range (0, j_shape[0]):
    for j in range (0, j_shape[1]):
        t_mse = t_mse + (j_test_df.values[i, j] - predict_test[i, j]) * (j_test_df.values[i, j] - predict_test[i, j])
        if predict_test[i, j] != 99:
            t_number =t_number + 1
t_mse = t_mse/t_number


In [119]:
print(t_mse,t_number)

23.47539651314553 701
